## premise: Grid search for clustering solution

via skleanr pipeline and grid search

utilize a distribution across known bins, to normalize the output variable

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
import sklearn.metrics
import numpy as np
from sklearn import preprocessing

In [15]:
#load pipeline 1 csv and prep for clustering
m2_pipeline = pd.read_csv('pipeline1.csv')
#change is surge price rate of change per observation, change.1 is precursor
#sum_change is surge sum_change per surge, and surge_area is surge alone
keepable = ['precursor_buy_cap_pct_change', 
            'precursor_ask_cap_pct_change',
            'precursor_bid_vol_pct_change', 
            'precursor_ask_vol_pct_change', 'change.1',
            'surge_targets_met_pct']

# Normalize the 'surge_targets_met_pct' column
x = m2_pipeline[['surge_targets_met_pct']].values.astype(float)
m2_pipeline = m2_pipeline[keepable]
m2_pipeline = m2_pipeline.dropna()
print(m2_pipeline.isna().sum(axis=1).astype(bool).sum())
m2_pipeline = m2_pipeline.astype('float')
m2_pipeline.dtypes

0


precursor_buy_cap_pct_change    float64
precursor_ask_cap_pct_change    float64
precursor_bid_vol_pct_change    float64
precursor_ask_vol_pct_change    float64
change.1                        float64
surge_targets_met_pct           float64
dtype: object

## using predefined bins for discrete clustering

In [19]:
# bins = [float(f"{x:.2f}") for x in range(-10, 11)]
bins = [x * 0.2 for x in range(-10, 11)]
bins

[-2.0,
 -1.8,
 -1.6,
 -1.4000000000000001,
 -1.2000000000000002,
 -1.0,
 -0.8,
 -0.6000000000000001,
 -0.4,
 -0.2,
 0.0,
 0.2,
 0.4,
 0.6000000000000001,
 0.8,
 1.0,
 1.2000000000000002,
 1.4000000000000001,
 1.6,
 1.8,
 2.0]

In [20]:
#model 

# Bin the values in the 'surge_targets_met_pct' column
# bins = [-float('inf'), -8.5, -5.64, 0, 0.25, 0.35, 0.4, 0.5, 0.75, 1.0, 2.0, 3.0, 4.0, 5.0, 5.64, float('inf')]
# bins = rounded_values
# labels = rounded_values #['< -8.5', '-8.5 to -5.64', '-5.64 to 0', '0 to 0.25', '0.25 to 0.35', '0.35 to 0.4', '0.4 to 0.5', '0.5 to 0.75', '0.75 to 1', '1 to 2', '2 to 3', '3 to 4', '4 to 5', '5 to 5.64', '> 5.64']
m2_pipeline['bin'] = pd.cut(m2_pipeline['surge_targets_met_pct'], bins=bins  )#, labels=labels)

# Display the binned data
print(m2_pipeline['bin'].value_counts())
#implement
# transformer = preprocessing.FunctionTransformer(
#     pd.cut, kw_args={'bins': bins, 'retbins': False}  #'labels': labels, 
# )
# transformer.fit_transform(x)

(-0.2, 0.0]     1922
(-0.4, -0.2]    1043
(-0.6, -0.4]     590
(0.0, 0.2]       489
(-0.8, -0.6]     396
(-1.0, -0.8]     235
(0.2, 0.4]       201
(-1.2, -1.0]     157
(-1.4, -1.2]      92
(-1.6, -1.4]      71
(-1.8, -1.6]      61
(0.4, 0.6]        60
(-2.0, -1.8]      51
(0.6, 0.8]        32
(0.8, 1.0]        15
(1.0, 1.2]        13
(1.2, 1.4]        10
(1.4, 1.6]         2
(1.6, 1.8]         2
(1.8, 2.0]         1
Name: bin, dtype: int64


In [21]:
m2_pipeline['bin']

1        (-0.2, 0.0]
2        (-0.2, 0.0]
3        (-0.2, 0.0]
4        (-0.2, 0.0]
5       (-0.4, -0.2]
            ...     
5631     (-0.2, 0.0]
5632     (-0.2, 0.0]
5633     (-0.2, 0.0]
5634     (-0.2, 0.0]
5635     (-0.2, 0.0]
Name: bin, Length: 5634, dtype: category
Categories (20, interval[float64, right]): [(-2.0, -1.8] < (-1.8, -1.6] < (-1.6, -1.4] < (-1.4, -1.2] ... (1.2, 1.4] < (1.4, 1.6] < (1.6, 1.8] < (1.8, 2.0]]

In [22]:
m2_pipeline.columns

Index(['precursor_buy_cap_pct_change', 'precursor_ask_cap_pct_change',
       'precursor_bid_vol_pct_change', 'precursor_ask_vol_pct_change',
       'change.1', 'surge_targets_met_pct', 'bin'],
      dtype='object')

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
# from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, SpectralClustering, Birch
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering,\
MeanShift, AffinityPropagation, DBSCAN, OPTICS, Birch
from sklearn.metrics import silhouette_score
# Splitting the dataframe into features and labels
X = m2_pipeline.drop(columns=['bin'])
y = m2_pipeline['bin']
# Performing the test/train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
def silhouette_scorer(estimator, X, y=None):
    labels = estimator.fit_predict(X)
    score = silhouette_score(X, labels)
    return score
# Defining the parameter grid for GridSearchCV
param_grid = {'n_clusters': [7,8,9,10,11,12,13,14]}  #'algorithm': ['auto', 'full', 'elkan']
clustering_models = [
    ('KMeans', KMeans()),
    ('SpectralClustering', SpectralClustering()),
    ('Birch', Birch()),
('Hierarchical',AgglomerativeClustering())]
# Performing GridSearchCV for each clustering model
for model_name, model in clustering_models:
    grid_search = GridSearchCV(model, param_grid, scoring=silhouette_scorer)
    grid_search.fit(X_train)
    # Evaluating the best model based on silhouette score
    best_model = grid_search.best_estimator_
    best_score = grid_search.best_score_
    print(f'{model_name}:')
    print(f'Best parameters: {grid_search.best_params_}')
    print(f'Silhouette score (train): {best_score:.4f}')
    print(f'Silhouette score (test): {best_score:.4f}')

/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
 

KMeans:
Best parameters: {'n_clusters': 9}
Silhouette score (train): 0.4830
Silhouette score (test): 0.4830


/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectra

SpectralClustering:
Best parameters: {'n_clusters': 7}
Silhouette score (train): 0.6975
Silhouette score (test): 0.6975


/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/cluster/_birch.py:726: ConvergenceWarning: Number of subclusters found (7) by BIRCH is less than (8). Decrease the threshold.
  warnings.warn(
/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/cluster/_birch.py:726: ConvergenceWarning: Number of subclusters found (7) by BIRCH is less than (8). Decrease the threshold.
  warnings.warn(
/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/cluster/_birch.py:726: ConvergenceWarning: Number of subclusters found (7) by BIRCH is less than (9). Decrease the threshold.
  warnings.warn(
/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/cluster/_birch.py:726: ConvergenceWarning: Number of subclusters found (8) by BIRCH is less than (9). Decrease the threshold.
  warnings.warn(
/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/cluster/_birch.py:726: ConvergenceWarning: Number of subclusters found (8) by BIRCH

Birch:
Best parameters: {'n_clusters': 7}
Silhouette score (train): 0.6072
Silhouette score (test): 0.6072


In [42]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
# from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, SpectralClustering, Birch
from sklearn.cluster import MeanShift, AffinityPropagation, DBSCAN, OPTICS
from sklearn.metrics import silhouette_score
# Splitting the dataframe into features and labels
X = m2_pipeline.drop(columns=['bin'])
y = m2_pipeline['bin']
# Performing the test/train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
def silhouette_scorer(estimator, X, y=None):
    labels = estimator.fit_predict(X)
    score = silhouette_score(X, labels)
    return score
# Defining the parameter grid for GridSearchCV
# meanshift = MeanShift(bandwidth=None, seeds=None, bin_seeding=False, min_bin_freq=1, cluster_all=True, n_jobs=None, max_iter=300)
# affinity = AffinityPropagation( damping=0.5, max_iter=200, convergence_iter=15, copy=True, preference=None, affinity='euclidean', verbose=False, random_state=None)
dbscan = DBSCAN(eps=0.5,  min_samples=5, metric='euclidean', metric_params=None, algorithm='auto', leaf_size=30, p=None, n_jobs=None)
optics = OPTICS( min_samples=5, max_eps=3, metric='minkowski', p=2, metric_params=None, cluster_method='xi', eps=None, xi=0.05, predecessor_correction=True, min_cluster_size=None, algorithm='auto', leaf_size=30, memory=None, n_jobs=None)

param_grid = {}  #'algorithm': ['auto', 'full', 'elkan']
clustering_models = [
    ('Meanshift',  MeanShift( bandwidth=None, seeds=None, bin_seeding=False, min_bin_freq=1, cluster_all=True, n_jobs=None, max_iter=300)),
    ('AffinityPropagation', AffinityPropagation( damping=0.5, max_iter=200, convergence_iter=15, copy=True, preference=None, affinity='euclidean', verbose=False, random_state=42)),
    ('DBSCAN', DBSCAN(eps=0.5,  min_samples=5, metric='euclidean', metric_params=None, algorithm='auto', leaf_size=30, p=None, n_jobs=None)),
('OPTICS', OPTICS( min_samples=5, max_eps=3, metric='minkowski', p=2, metric_params=None, cluster_method='xi', eps=None, xi=0.05, predecessor_correction=True, min_cluster_size=None, algorithm='auto', leaf_size=30, memory=None, n_jobs=None))]
# Performing GridSearchCV for each clustering model
for model_name, model in clustering_models:
    grid_search = GridSearchCV(model, param_grid, scoring=silhouette_scorer)
    grid_search.fit(X_train)
    # Evaluating the best model based on silhouette score
    best_model = grid_search.best_estimator_
    best_score = grid_search.best_score_
    print(f'{model_name}:')
    print(f'Best parameters: {grid_search.best_params_}')
    print(f'Silhouette score (train): {best_score:.4f}')
    print(f'Silhouette score (test): {best_score:.4f}')

Meanshift:
Best parameters: {}
Silhouette score (train): 0.5083
Silhouette score (test): 0.5083


/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/cluster/_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(
/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/cluster/_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(
/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/cluster/_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(
/home/stefan/Desktop/GRUS/caret1/lib/python3.8/site-packages/sklearn/cluster/_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(
/home/stefan/Desktop/GRU

AffinityPropagation:
Best parameters: {}
Silhouette score (train): 0.3188
Silhouette score (test): 0.3188
DBSCAN:
Best parameters: {}
Silhouette score (train): 0.8522
Silhouette score (test): 0.8522
OPTICS:
Best parameters: {}
Silhouette score (train): -0.3168
Silhouette score (test): -0.3168


## notes 

1. grid search notes on [sklearn](https://scikit-learn.org/stable/modules/grid_search.html#grid-search)
2. sklearn pipeline [guide](https://scikit-learn.org/stable/modules/compose.html#pipeline)